In [57]:
import findspark
findspark.init()
import pandas as pd
import pyspark as ps
import warnings
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType, IntegerType, FloatType, DoubleType, StructType, StructField
import csv

In [62]:
sc = SparkContext()
sqlContext = SQLContext(sc)

In [61]:
#sc.stop()

In [63]:
schema = StructType([
    StructField("col_01", StringType()),
    StructField("col_02", StringType()),
    StructField("col_03", StringType()),
    StructField("col_04", StringType()),
    StructField("col_05", IntegerType()),
    StructField("col_06", IntegerType()),
    StructField("col_07", IntegerType()),
    StructField("col_08", IntegerType()),
    
])

In [67]:
data = []
with open('small_tweets.csv', 'r' ) as doc:
    reader = csv.DictReader(doc)
    for line in reader:
        data.append(line)

df = sc.parallelize(data).toDF()

In [64]:
#data = sqlContext.read.format('csv') \
#.options(header='true', schema=schema) \
#.load('small_tweets.csv')

In [69]:
df.show()

+----------+----------------+---------------+-------------+---------------+-------------------+--------------------+--------+
|      date|favourites_count|followers_count|retweet_count|    screen_name|             source|                text|verified|
+----------+----------------+---------------+-------------+---------------+-------------------+--------------------+--------+
|2020-03-22|              37|            125|            0|     ChirpyPaul|   Twitter for iPad|I have overhead p...|       0|
|2020-03-23|              99|             75|            0|   CreeperErock|          Instagram|Check in with you...|       0|
|2020-03-13|            2398|           1121|            0|  UsamaAlshaibi| Twitter for iPhone|Grocery stores ar...|       0|
|2020-03-30|             652|            152|            0|        jpcybil| Twitter for iPhone|A day in the life...|       0|
|2020-03-28|            1622|            263|            0| basu_subhashis|Twitter for Android|Distribution of f...|  

In [70]:
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- favourites_count: string (nullable = true)
 |-- followers_count: string (nullable = true)
 |-- retweet_count: string (nullable = true)
 |-- screen_name: string (nullable = true)
 |-- source: string (nullable = true)
 |-- text: string (nullable = true)
 |-- verified: string (nullable = true)



In [71]:
flag1 = df.withColumn("favourites_count", df["favourites_count"].cast(IntegerType()))

In [72]:
flag2 = flag1.withColumn("retweet_count", flag1["retweet_count"].cast(IntegerType()))

In [73]:
flag3 = flag2.withColumn("followers_count", flag2["followers_count"].cast(IntegerType()))

In [74]:
dfnew = flag3.withColumn("verified", flag3["verified"].cast(IntegerType()))

In [75]:
dfnew.printSchema()

root
 |-- date: string (nullable = true)
 |-- favourites_count: integer (nullable = true)
 |-- followers_count: integer (nullable = true)
 |-- retweet_count: integer (nullable = true)
 |-- screen_name: string (nullable = true)
 |-- source: string (nullable = true)
 |-- text: string (nullable = true)
 |-- verified: integer (nullable = true)



In [76]:
dfnew.show(5)

+----------+----------------+---------------+-------------+--------------+-------------------+--------------------+--------+
|      date|favourites_count|followers_count|retweet_count|   screen_name|             source|                text|verified|
+----------+----------------+---------------+-------------+--------------+-------------------+--------------------+--------+
|2020-03-22|              37|            125|            0|    ChirpyPaul|   Twitter for iPad|I have overhead p...|       0|
|2020-03-23|              99|             75|            0|  CreeperErock|          Instagram|Check in with you...|       0|
|2020-03-13|            2398|           1121|            0| UsamaAlshaibi| Twitter for iPhone|Grocery stores ar...|       0|
|2020-03-30|             652|            152|            0|       jpcybil| Twitter for iPhone|A day in the life...|       0|
|2020-03-28|            1622|            263|            0|basu_subhashis|Twitter for Android|Distribution of f...|       0|


In [22]:
df = df.fillna({'favourites_count':'0'}).show(5)


+--------------------+--------------------+--------------------+------+----------------+-------------+---------------+--------+
|                date|         screen_name|                text|source|favourites_count|retweet_count|followers_count|verified|
+--------------------+--------------------+--------------------+------+----------------+-------------+---------------+--------+
|          2020-03-22|          ChirpyPaul|I have overhead p...|  null|               0|         null|           null|    null|
|Holiday caravan p...| themed venues et...|                null|  null|               0|         null|           null|    null|
|       Stay at home.|                null|                null|  null|               0|         null|           null|    null|
|#Covid_19 https:/...|    Twitter for iPad|                37.0|   0.0|             125|            0|           null|    null|
|          2020-03-23|        CreeperErock|Check in with you...|  null|               0|         null|  

In [23]:
df.show(5)

+--------------------+--------------------+--------------------+------+----------------+-------------+---------------+--------+
|                date|         screen_name|                text|source|favourites_count|retweet_count|followers_count|verified|
+--------------------+--------------------+--------------------+------+----------------+-------------+---------------+--------+
|          2020-03-22|          ChirpyPaul|I have overhead p...|  null|            null|         null|           null|    null|
|Holiday caravan p...| themed venues et...|                null|  null|            null|         null|           null|    null|
|       Stay at home.|                null|                null|  null|            null|         null|           null|    null|
|#Covid_19 https:/...|    Twitter for iPad|                37.0|   0.0|             125|            0|           null|    null|
|          2020-03-23|        CreeperErock|Check in with you...|  null|            null|         null|  

In [68]:
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- screen_name: string (nullable = true)
 |-- text: string (nullable = true)
 |-- source: string (nullable = true)
 |-- favourites_count: integer (nullable = true)
 |-- retweet_count: string (nullable = true)
 |-- followers_count: string (nullable = true)
 |-- verified: string (nullable = true)



In [48]:
from pyspark.sql.functions import col
data.groupBy("country_code") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+---------------+------+
|   country_code| count|
+---------------+------+
|           null|121455|
|          False| 28823|
|           True|  2952|
|             US|  1530|
|             GB|   464|
|            0.0|   334|
|             CA|   170|
|             IN|   160|
|             IE|    71|
|          16640|    63|
|          16641|    62|
|             AU|    60|
|            1.0|    54|
|          16744|    43|
|             NG|    42|
|Twitter Web App|    41|
|             ZA|    36|
|          16580|    32|
|            2.0|    32|
|             PH|    24|
+---------------+------+
only showing top 20 rows



In [10]:
drop_list = [['_c0',
 'screen_name',
 'source',
 'favourites_count',
 'retweet_count',
 'country_code',
 'place_full_name',
 'followers_count',
 'verified',
 'lang']]

In [40]:
data.columns

['_c0',
 'created_at',
 'screen_name',
 'text',
 'source',
 'favourites_count',
 'retweet_count',
 'country_code',
 'place_full_name',
 'followers_count',
 'verified',
 'lang']